In [17]:
# Conta a quantia de artigos, filtra as duplicatas mantendo artigos sem DOI
import pandas as pd
df = pd.read_csv('snowballing02_initial.csv')
print(len(df)) 


df_non_null = df[df['DOI'].notna()]
    
# Identifica duplicatas apenas entre os valores não nulos
duplicatas = df_non_null['DOI'].duplicated(keep=False)

# Quantidade de duplicatas não nulas
num_duplicatas = duplicatas.sum()
print(f"Quantidade de duplicatas (não nulas) encontradas na coluna 'DOI': {num_duplicatas}")

# Remove duplicatas não nulas (mantendo a primeira ocorrência)
df_non_null_cleaned = df_non_null.drop_duplicates(subset='DOI', keep='first')

# Combina as linhas sem duplicatas não nulas com as linhas onde 'DOI' é nulo
df_cleaned = pd.concat([df_non_null_cleaned, df[df['DOI'].isna()]])

# Salva o DataFrame limpo em um novo arquivo CSV, se necessário
print(len(df_cleaned))
df_cleaned.to_csv('snowballing03_no_dupplicates.csv', index=False)

1958
Quantidade de duplicatas (não nulas) encontradas na coluna 'DOI': 51
1931


In [2]:
# Eliminação automática a partir de critérios de inclusão simples
import pandas as pd

df = pd.read_csv('snowballing03_no_dupplicates.csv')

keywords_title = ['radiomics', 'network', 'redes', 'radiômica', 'oncology', 'cancer', 'feature selection']
keywords_abstract = ['radiomics', 'network', 'redes', 'radiômica', 'oncology', 'cancer', 'feature selection']
exclusao_title = [
    'review', 'meta-analysis', 'protocol', 'survey', 'case study',
    'genomics', 'proteomics', 'cardiology', 'neurology', 'pediatrics',
    'epidemiology', 'public health', 'prevention', 'screening', 'non-cancer',
    'stroke', 'surgery', 'cardiac',
]

def verify_year(year):
    if pd.isna(year):  # Verifica se o texto é NaN
        return False
    # Verifica se alguma das palavras-chave de exclusão está no texto

    return year >= 2020
    return any(keyword.lower() in texto.lower() for keyword in exclusao_title)

def contem_palavra_exclusao(texto):
    if pd.isna(texto):  # Verifica se o texto é NaN
        return False
    # Verifica se alguma das palavras-chave de exclusão está no texto
    return any(keyword.lower() in texto.lower() for keyword in exclusao_title)

def contain_at_least(texto, type):
    if type == 'Title':
        if pd.isna(texto):  # Verifica se o texto é NaN
            return False
        texto = str(texto)
        # Verifica se ao menos uma das palavras-chave está no texto
        return any(keyword.lower() in texto.lower() for keyword in keywords_title)
    texto = str(texto)
    return any(keyword.lower() in texto.lower() for keyword in keywords_abstract)

# Mantém apenas linhas que contenham uma das keywords ao menos
df_filtrado = df[
    df.apply(lambda row: contain_at_least(row['Title'], 'Title') or 
                        contain_at_least(row['Abstract'], 'Abstract'), axis=1)
]
print(len(df_filtrado))
df_filtrado = df_filtrado[
    ~df_filtrado.apply(lambda row: contem_palavra_exclusao(row['Title']), axis=1)
]

# Exibir a quantidade de artigos restantes
print(f"Número de artigos restantes após o filtro personalizado: {len(df_filtrado)}")

df_filtrado = df_filtrado[
    df_filtrado.apply(lambda row: verify_year(row['Year']), axis=1)
]

print(f"Após descartar anos menores que 2020: {len(df_filtrado)}")

# Salvar o novo DataFrame filtrado em um arquivo CSV
df_filtrado.to_csv('snowballing04_simple_filter.csv', index=False)

1713
Número de artigos restantes após o filtro personalizado: 1457
Após descartar anos menores que 2020: 1235
